In [1]:
from IPython.display import Image

- DAG: Directed Acyclic Graph
    - 有向无环图；
        - 环：means cycle back；
    - formalized workflow for transitioning data
    - 一个节点可以有多个出边，即有多条 outgoing 的路径；

In [3]:
Image(url='./dag.png', width=300)

### prompt template

- https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html

In [1]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)